## Milestone 2 

In [15]:
# load the dataset 
import pandas as pd

# Pfad ggf. anpassen
wc_path = "C:/Desktop/ThKoeln/WI2526/dis08/projekt/WorldCups.csv"

wc = pd.read_csv(wc_path)
wc.head()


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607


In [16]:
# save all world cup years in a list 
years = sorted(wc["Year"].unique())
print(years)


[np.int64(1930), np.int64(1934), np.int64(1938), np.int64(1950), np.int64(1954), np.int64(1958), np.int64(1962), np.int64(1966), np.int64(1970), np.int64(1974), np.int64(1978), np.int64(1982), np.int64(1986), np.int64(1990), np.int64(1994), np.int64(1998), np.int64(2002), np.int64(2006), np.int64(2010), np.int64(2014)]


In [17]:
# save all top 3 countrys in a list 
cols = ["Winner", "Runners-Up", "Third"] 

countries_series = pd.concat([wc[c] for c in cols], axis=0)

# NaN rausfiltern und Duplikate entfernen
countries = sorted(countries_series.dropna().unique())
countries


['Argentina',
 'Austria',
 'Brazil',
 'Chile',
 'Croatia',
 'Czechoslovakia',
 'England',
 'France',
 'Germany',
 'Germany FR',
 'Hungary',
 'Italy',
 'Netherlands',
 'Poland',
 'Portugal',
 'Spain',
 'Sweden',
 'Turkey',
 'USA',
 'Uruguay']

In [18]:
# dictonary with years and top 3 countrys 

top3_by_year = {}

for _, row in wc.iterrows():
    year = int(row["Year"])
    countries = []
    for col in ["Winner", "Runners-Up", "Third"]:
        if pd.notna(row[col]):
            countries.append(row[col])
    top3_by_year[year] = countries

top3_by_year


{1930: ['Uruguay', 'Argentina', 'USA'],
 1934: ['Italy', 'Czechoslovakia', 'Germany'],
 1938: ['Italy', 'Hungary', 'Brazil'],
 1950: ['Uruguay', 'Brazil', 'Sweden'],
 1954: ['Germany FR', 'Hungary', 'Austria'],
 1958: ['Brazil', 'Sweden', 'France'],
 1962: ['Brazil', 'Czechoslovakia', 'Chile'],
 1966: ['England', 'Germany FR', 'Portugal'],
 1970: ['Brazil', 'Italy', 'Germany FR'],
 1974: ['Germany FR', 'Netherlands', 'Poland'],
 1978: ['Argentina', 'Netherlands', 'Brazil'],
 1982: ['Italy', 'Germany FR', 'Poland'],
 1986: ['Argentina', 'Germany FR', 'France'],
 1990: ['Germany FR', 'Argentina', 'Italy'],
 1994: ['Brazil', 'Italy', 'Sweden'],
 1998: ['France', 'Brazil', 'Croatia'],
 2002: ['Brazil', 'Germany', 'Turkey'],
 2006: ['Italy', 'France', 'Germany'],
 2010: ['Spain', 'Netherlands', 'Germany'],
 2014: ['Germany', 'Argentina', 'Netherlands']}


### Country name normalization and handling historical countries

In the FIFA World Cup dataset, some historical country names or designations appear that do not exist as separate entries in the population dataset (for example **"Germany FR"** for the Federal Republic of Germany before reunification or **"Czechoslovakia"**). In order to merge both datasets, we apply the following pragmatic assumptions:

- **"Germany FR" → "Germany"**  
  For all entries labeled "Germany FR" we use the population figures of the modern entry "Germany". The World Cup participation in these years refers to the Federal Republic of Germany (West Germany), while our population dataset only provides a single entry for the unified Germany. Using "Germany" as an approximation is therefore a practical but transparent simplification.

- **"Czechoslovakia" → sum of "Czech Republic" + "Slovakia"**  
  For "Czechoslovakia" we approximate the historical population by summing the population of the present-day states "Czech Republic" and "Slovakia" (for the respective year). This assumption is explicitly documented and will be taken into account when interpreting the results.

These mapping decisions are conscious simplifications. They allow us to automatically merge the datasets and answer our research questions without having to reconstruct historically exact population data for every state. We document these assumptions in our methodology so that the limitations of the data are clear.


In [19]:

# Normalize historical country names to match the population dataset
name_mapping = {
    "Germany FR": "Germany"
    # Additional mappings can be added here if needed, e.g.:
    # "USA": "United States",
    # "England": "United Kingdom",
}

# Create a normalized version of the top3_by_year dictionary
# This assumes that `top3_by_year` has already been defined earlier in the notebook.
top3_by_year_normalized = {
    year: [name_mapping.get(country, country) for country in countries]
    for year, countries in top3_by_year.items()
}

top3_by_year_normalized


{1930: ['Uruguay', 'Argentina', 'USA'],
 1934: ['Italy', 'Czechoslovakia', 'Germany'],
 1938: ['Italy', 'Hungary', 'Brazil'],
 1950: ['Uruguay', 'Brazil', 'Sweden'],
 1954: ['Germany', 'Hungary', 'Austria'],
 1958: ['Brazil', 'Sweden', 'France'],
 1962: ['Brazil', 'Czechoslovakia', 'Chile'],
 1966: ['England', 'Germany', 'Portugal'],
 1970: ['Brazil', 'Italy', 'Germany'],
 1974: ['Germany', 'Netherlands', 'Poland'],
 1978: ['Argentina', 'Netherlands', 'Brazil'],
 1982: ['Italy', 'Germany', 'Poland'],
 1986: ['Argentina', 'Germany', 'France'],
 1990: ['Germany', 'Argentina', 'Italy'],
 1994: ['Brazil', 'Italy', 'Sweden'],
 1998: ['France', 'Brazil', 'Croatia'],
 2002: ['Brazil', 'Germany', 'Turkey'],
 2006: ['Italy', 'France', 'Germany'],
 2010: ['Spain', 'Netherlands', 'Germany'],
 2014: ['Germany', 'Argentina', 'Netherlands']}


### Inspecting and aligning the world population dataset

In the next step, we want to understand which of our World Cup top-3 countries and years are already covered by the existing population dataset and where values are missing and need to be augmented.

**Key design decisions for now:**

- We treat **"Germany"** in the World Cup data as directly compatible with **"Germany"** in the population dataset. We do **not** distinguish between historical West/East Germany in this project.
- For **"Czechoslovakia"** we currently accept that we do *not* have a corresponding entry in the population dataset. For now, these rows will simply end up with missing population values (NaN) and can either be ignored or handled separately during analysis.
- This keeps the focus on building a working data pipeline rather than perfectly reconstructing all historical population values.

The goal of the following code is:

1. Load the `world_population.csv` file.
2. Transform it from the wide format (columns like `1970 Population`, `1980 Population`, …) into a long format with columns `Country`, `Year`, `Population`.
3. Build a DataFrame of all `(Year, Country)` pairs from our `top3_by_year` dictionary.
4. Merge these two structures to see which combinations already have population values and which are missing.


In [20]:
# --- 1. Load the world population dataset ---
# Adjust the path if your CSV file is stored in a different folder.
pop_path = "world_population.csv"
pop = pd.read_csv(pop_path)

# Rename to a simpler column name
pop = pop.rename(columns={"Country/Territory": "Country"})

# Inspect columns to identify the population year columns
pop.columns.tolist()

# --- 2. Transform to long format: Country, Year, Population ---
# We assume that the year-specific columns contain a 4-digit year in their name (e.g. "1970 Population").
year_columns = [col for col in pop.columns if any(char.isdigit() for char in col)]

# Melt the dataset into long format
pop_long = pop.melt(
    id_vars=["Country"],
    value_vars=year_columns,
    var_name="Year_raw",
    value_name="Population"
)

# Extract the numeric year from the column name, e.g. "1970 Population" -> 1970
pop_long["Year"] = pop_long["Year_raw"].str.extract(r"(\\d{4})").astype(float)
pop_long = pop_long.dropna(subset=["Year"])
pop_long["Year"] = pop_long["Year"].astype(int)

# Optional: drop rows where population is missing
pop_long = pop_long.dropna(subset=["Population"])

# --- 3. Build a DataFrame of all (Year, Country) pairs from top3_by_year ---
rows = []
for year, countries in top3_by_year.items():
    for country in countries:
        rows.append({"Year": year, "Country": country})

needed_pairs = pd.DataFrame(rows)

# --- 4. Merge to see which combinations have population values ---
needed_with_pop = needed_pairs.merge(
    pop_long[["Country", "Year", "Population"]],
    on=["Country", "Year"],
    how="left"
)

# Mark missing population values (these will need augmentation or can be excluded later)
needed_with_pop["missing_population"] = needed_with_pop["Population"].isna()

needed_with_pop.head(), needed_with_pop["missing_population"].value_counts()


(   Year         Country Population  missing_population
 0  1930         Uruguay        NaN                True
 1  1930       Argentina        NaN                True
 2  1930             USA        NaN                True
 3  1934           Italy        NaN                True
 4  1934  Czechoslovakia        NaN                True,
 missing_population
 True    60
 Name: count, dtype: int64)


### Fixing population melt and merge (ensuring 1970, 2010, etc. are correctly populated)

In this section we fix two important issues in the previous attempt:

1. **Year column selection**: We now select only columns that actually represent population for a specific year, e.g. `"1970 Population"`, `"2010 Population"`, etc. We avoid columns like `"Area (km²)"` or `"Density (per km²)"` that also contain digits.
2. **Year extraction**: We use the correct regular expression `r"(\\d{4})"` to extract the 4-digit year from column names such as `"1970 Population"`.

This ensures that population values for years like **1970** and **2010** are correctly available in the long-format population table and therefore also in the merged table of World Cup top-3 countries.


In [21]:
import pandas as pd

# --- (Optional) Rebuild top3_by_year from WorldCups.csv if it is not yet defined ---
try:
    top3_by_year
except NameError:
    wc_path = "WorldCups.csv"
    wc = pd.read_csv(wc_path)
    top3_by_year = {}
    for _, row in wc.iterrows():
        year = int(row["Year"])
        countries = []
        for col in ["Winner", "Runners-Up", "Third"]:
            if pd.notna(row[col]):
                countries.append(row[col])
        top3_by_year[year] = countries

# --- 1. Load the world population dataset ---
pop_path = "world_population.csv"
pop = pd.read_csv(pop_path)

# Normalize column name for the country
pop = pop.rename(columns={"Country/Territory": "Country"})

# --- 2. Select ONLY valid year population columns ---
year_columns = [
    col for col in pop.columns
    if col.endswith("Population") and col[:4].isdigit()
]

print("Selected year columns:", year_columns)

# --- 3. Transform to long format: Country, Year, Population ---
pop_long = pop.melt(
    id_vars=["Country"],
    value_vars=year_columns,
    var_name="Year_raw",
    value_name="Population"
)

# Extract numeric year, e.g. "1970 Population" -> 1970
pop_long["Year"] = pop_long["Year_raw"].str.extract(r"(\d{4})").astype(int)

# Drop rows without population values
pop_long = pop_long.dropna(subset=["Population"])

# --- 4. Build DataFrame with all (Year, Country) pairs from top3_by_year ---
rows = []
for year, countries in top3_by_year.items():
    for country in countries:
        rows.append({"Year": year, "Country": country})

needed_pairs = pd.DataFrame(rows)

# --- 4a. Normalize historical country names to match population dataset ---
name_mapping = {
    "Germany FR": "Germany",
    # here you could later add more, e.g.:
    # "USA": "United States",
    # "England": "United Kingdom",
}
needed_pairs["Country"] = needed_pairs["Country"].replace(name_mapping)

# --- 5. Merge to check which combinations have population values ---
needed_with_pop = needed_pairs.merge(
    pop_long[["Country", "Year", "Population"]],
    on=["Country", "Year"],
    how="left"
)

# Mark missing population values
needed_with_pop["missing_population"] = needed_with_pop["Population"].isna()

# Show all rows for 1970, 1990, and 2010 to verify that populations are present
print("\nTop-3 countries for 1970 with population:")
print(needed_with_pop[needed_with_pop["Year"] == 1970])

print("\nTop-3 countries for 1990 with population:")
print(needed_with_pop[needed_with_pop["Year"] == 1990])

print("\nTop-3 countries for 2010 with population:")
print(needed_with_pop[needed_with_pop["Year"] == 2010])



Selected year columns: ['2022 Population', '2020 Population', '2015 Population', '2010 Population', '2000 Population', '1990 Population', '1980 Population', '1970 Population']

Top-3 countries for 1970 with population:
    Year  Country  Population  missing_population
24  1970   Brazil  96369875.0               False
25  1970    Italy  53324036.0               False
26  1970  Germany  78294583.0               False

Top-3 countries for 1990 with population:
    Year    Country  Population  missing_population
39  1990    Germany  79370196.0               False
40  1990  Argentina  32637657.0               False
41  1990      Italy  56756561.0               False

Top-3 countries for 2010 with population:
    Year      Country  Population  missing_population
54  2010        Spain  46572772.0               False
55  2010  Netherlands  16617116.0               False
56  2010      Germany  81325090.0               False


## values which we need to get through scraping

In [22]:
missing_list = needed_with_pop[needed_with_pop["missing_population"] == True] \
    .sort_values(["Year", "Country"])[["Year", "Country"]]

missing_no_cz = missing_list[missing_list["Country"] != "Czechoslovakia"].copy()


print(missing_no_cz)
print(len(missing_no_cz))

    Year      Country
1   1930    Argentina
2   1930          USA
0   1930      Uruguay
5   1934      Germany
3   1934        Italy
8   1938       Brazil
7   1938      Hungary
6   1938        Italy
10  1950       Brazil
11  1950       Sweden
9   1950      Uruguay
14  1954      Austria
12  1954      Germany
13  1954      Hungary
15  1958       Brazil
17  1958       France
16  1958       Sweden
18  1962       Brazil
20  1962        Chile
21  1966      England
22  1966      Germany
23  1966     Portugal
27  1974      Germany
28  1974  Netherlands
29  1974       Poland
30  1978    Argentina
32  1978       Brazil
31  1978  Netherlands
34  1982      Germany
33  1982        Italy
35  1982       Poland
36  1986    Argentina
38  1986       France
37  1986      Germany
42  1994       Brazil
43  1994        Italy
44  1994       Sweden
46  1998       Brazil
47  1998      Croatia
45  1998       France
48  2002       Brazil
49  2002      Germany
50  2002       Turkey
52  2006       France
53  2006  

## Daten beschaffen

In [23]:
# Ausgangspunkt: fehlende Jahr-Land-Paare aus der vorherigen Analyse
missing_pairs = missing_no_cz.copy()

print("Anzahl fehlender Jahr-Land-Paare:", len(missing_pairs))
missing_pairs



Anzahl fehlender Jahr-Land-Paare: 49


,Year,Country
1,1930,Argentina
2,1930,USA
0,1930,Uruguay
5,1934,Germany
3,1934,Italy
8,1938,Brazil
7,1938,Hungary
6,1938,Italy
10,1950,Brazil
11,1950,Sweden


In [24]:
# Schritt 2: Überblick über die benötigten Jahre und Länder

years = missing_pairs["Year"].unique()
countries = missing_pairs["Country"].unique()

print("Benötigte Jahre:")
print(years)

print("\nBenötigte Länder:")
print(countries)

print("\nAnzahl Jahre:", len(years))
print("Anzahl Länder:", len(countries))


Benötigte Jahre:
[1930 1934 1938 1950 1954 1958 1962 1966 1974 1978 1982 1986 1994 1998
 2002 2006 2014]

Benötigte Länder:
['Argentina' 'USA' 'Uruguay' 'Germany' 'Italy' 'Brazil' 'Hungary' 'Sweden'
 'Austria' 'France' 'Chile' 'England' 'Portugal' 'Netherlands' 'Poland'
 'Croatia' 'Turkey']

Anzahl Jahre: 17
Anzahl Länder: 17


In [25]:
# Schritt 3: Country -> ISO3-Code mappen (für die World-Bank-API)

country_to_iso3 = {
    "Argentina": "ARG",
    "USA": "USA",
    "Uruguay": "URY",
    "Germany": "DEU",
    "Italy": "ITA",
    "Brazil": "BRA",
    "Hungary": "HUN",
    "Sweden": "SWE",
    "Austria": "AUT",
    "France": "FRA",
    "Chile": "CHL",
    "England": "GBR",   # World Bank nutzt GBR für United Kingdom
    "Portugal": "PRT",
    "Netherlands": "NLD",
    "Poland": "POL",
    "Croatia": "HRV",
    "Turkey": "TUR",
    "Spain": "ESP" 
}

missing_pairs["iso3"] = missing_pairs["Country"].map(country_to_iso3)

print(missing_pairs)


    Year      Country iso3
1   1930    Argentina  ARG
2   1930          USA  USA
0   1930      Uruguay  URY
5   1934      Germany  DEU
3   1934        Italy  ITA
8   1938       Brazil  BRA
7   1938      Hungary  HUN
6   1938        Italy  ITA
10  1950       Brazil  BRA
11  1950       Sweden  SWE
9   1950      Uruguay  URY
14  1954      Austria  AUT
12  1954      Germany  DEU
13  1954      Hungary  HUN
15  1958       Brazil  BRA
17  1958       France  FRA
16  1958       Sweden  SWE
18  1962       Brazil  BRA
20  1962        Chile  CHL
21  1966      England  GBR
22  1966      Germany  DEU
23  1966     Portugal  PRT
27  1974      Germany  DEU
28  1974  Netherlands  NLD
29  1974       Poland  POL
30  1978    Argentina  ARG
32  1978       Brazil  BRA
31  1978  Netherlands  NLD
34  1982      Germany  DEU
33  1982        Italy  ITA
35  1982       Poland  POL
36  1986    Argentina  ARG
38  1986       France  FRA
37  1986      Germany  DEU
42  1994       Brazil  BRA
43  1994        Italy  ITA
4

In [26]:
import requests

# Indikator für "Population, total" bei der World Bank
indicator = "SP.POP.TOTL"

# Alle ISO3-Codes, die wir brauchen
iso_list = missing_pairs["iso3"].unique()
countries_str = ";".join(iso_list)

# kleinster und größter Jahrgang aus missing_pairs
year_min = int(missing_pairs["Year"].min())
year_max = int(missing_pairs["Year"].max())

print("Länder (ISO3):", countries_str)
print("Jahre:", year_min, "bis", year_max)

# API-URL
url = f"https://api.worldbank.org/v2/country/{countries_str}/indicator/{indicator}"

# Parameter für die Anfrage
params = {
    "format": "json",
    "date": f"{year_min}:{year_max}",
    "per_page": 20000
}

# Anfrage abschicken
response = requests.get(url, params=params)

# Statuscode anzeigen (200 = OK)
print("HTTP-Statuscode:", response.status_code)

# Antwort als JSON einlesen
data = response.json()

# Nur ganz grob die Struktur anzeigen
print("Typ von data:", type(data))
print("Anzahl Elemente in data:", len(data))


Länder (ISO3): ARG;USA;URY;DEU;ITA;BRA;HUN;SWE;AUT;FRA;CHL;GBR;PRT;NLD;POL;HRV;TUR
Jahre: 1930 bis 2014
HTTP-Statuscode: 200
Typ von data: <class 'list'>
Anzahl Elemente in data: 2


In [27]:
# Schritt 5: Die eigentlichen Datensätze aus dem JSON ziehen und als DataFrame anschauen

records = data[1]  # hier stehen die einzelnen Länder-Jahr-Werte drin

pop_df_raw = pd.json_normalize(records)

pop_df_raw


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,ARG,2014,43024071,,,0,SP.POP.TOTL,"Population, total",AR,Argentina
1,ARG,2013,42582455,,,0,SP.POP.TOTL,"Population, total",AR,Argentina
2,ARG,2012,42161721,,,0,SP.POP.TOTL,"Population, total",AR,Argentina
3,ARG,2011,41730660,,,0,SP.POP.TOTL,"Population, total",AR,Argentina
4,ARG,2010,41288694,,,0,SP.POP.TOTL,"Population, total",AR,Argentina
...,...,...,...,...,...,...,...,...,...,...
930,USA,1964,191889000,,,0,SP.POP.TOTL,"Population, total",US,United States
931,USA,1963,189242000,,,0,SP.POP.TOTL,"Population, total",US,United States
932,USA,1962,186538000,,,0,SP.POP.TOTL,"Population, total",US,United States
933,USA,1961,183691000,,,0,SP.POP.TOTL,"Population, total",US,United States


In [28]:
# DataFrame als CSV speichern
pop_df_raw.to_csv("worldbank_population_raw.csv", index=False)
